# Food Bank Problem

In [1]:
import sys
import numpy as np
import plotly.express as px
import pandas as pd
import scipy.optimize as optimization

## OPT - Waterfilling

In [2]:
## Water-filling Algorithm for sorted demands
def waterfilling_sorted(d,b):
    n = np.size(d)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        equal_allocation = bundle_remaining/(n-i)
        if d[i]<equal_allocation:
            allocations[i] = bundle_remaining if i==n-1 else d[i]
        else:
            allocations[i] = equal_allocation
        bundle_remaining -= allocations[i]
    return allocations

In [3]:
## Water-filling Algorithm for general demands
def waterfilling(d,b):
    n = np.size(d)
    sorted_indices = np.argsort(d)
    sorted_demands = np.sort(d)
    sorted_allocations = waterfilling_sorted(sorted_demands, b)
    allocations = np.zeros(n)
    for i in range(n):
        allocations[sorted_indices[i]] = sorted_allocations[i]
    return allocations

In [4]:
## Tests
assert list(waterfilling(np.zeros(0), 5)) == []
assert list(waterfilling(np.array([1,2,3,4]), 10)) == [1,2,3,4]
assert list(waterfilling(np.array([3,4,1,2]), 10)) == [3,4,1,2]
assert list(waterfilling(np.array([1,2,3,4]), 8)) == [1,2,2.5,2.5]
assert list(waterfilling(np.array([3,1,4,2]), 8)) == [2.5,1,2.5,2]
assert list(waterfilling(np.array([3,6,5,6]), 8)) == [2,2,2,2]

In [92]:
# Calculates the Bayes Optimal solution to the problem by solving the dynamic programming
# Note that this method was specified for the demand distribution where it is 1 with probability 1/2
# and 2 with probability 1/2
def bayes_opt(n, budget, grid_size):
    
    # Creates a discretization of the interval [0, budget] with bucket_size of grid_size
    b_grid = np.arange(0, budget+grid_size, grid_size)

    # Defines empty arrays for the estimates
    Q_vals = np.ones((len(b_grid), len(b_grid), 2, n+1))*(-100000)
    
    # Q_vals[b,a,d,h] will be the expected objective value for having a budget b, giving amount a to town h, where 
    # town h has a demand of d
    
    V_vals = np.zeros((len(b_grid), 2, n+1))
    # V_vals[b,d,h] will be the expected objective value for having a budget b where town h has a demand of d.
    
    # loops over each town backwards (as the last town is easy to solve)
    for h in np.arange(n, -1, -1):
        index_b = 1
        for b in b_grid[1:]: # loops over each budget in the grid
            index_a = 1
            for a in b_grid[1:]: # loops over each potential allocation in the grid
                
                if a <= b and a != 0: # Cannot give out more food than the budget, and cannot give out zero food
                    
                    # calculates index in the array for the remaining budget
                    remaining_budget = b-a
                    new_index = int(np.floor(remaining_budget / grid_size))
                    
                    if h == n:
                        # This town is the last town, so the objective is just the value for the allocation of a
                        # when their demand is 0 (corresponding to demand of 1) or 1 (corresponding to a demand of 2)
                        Q_vals[index_b, index_a, 0, h] = np.log(min(a,1))
                        Q_vals[index_b, index_a, 1, h] = np.log(min(a/2,1))
                    else:
                        # Otherowise they get the same immediate increase in objective value,
                        # but the future objective is the value for the next town and so on.
                        Q_vals[index_b, index_a, 0, h] = np.log(min(a,1)) + (1/2)*V_vals[new_index,0,h+1] + (1/2)*V_vals[new_index,1,h+1]
                        Q_vals[index_b, index_a, 1, h] = np.log(min(a/2,1)) + (1/2)*V_vals[new_index,0,h+1] + (1/2)*V_vals[new_index,1,h+1]
                index_a += 1
            # Update the V_function as the maximum of Q.  Note that we limit the bounds in the max
            # to give out some portion of food, and at most the budget
            V_vals[index_b,0,h] = max(Q_vals[index_b,1:(index_b+1),0,h])
            V_vals[index_b,1,h] = max(Q_vals[index_b,1:(index_b+1),1,h])
            index_b += 1
    return Q_vals

## Online Algorithms

In [27]:
## Online Water-filling taking minimum of realized demand and predetermeined allocation
def waterfilling_online_1(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    prior_allocations_assignment = waterfilling(demands_predicted,b)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        allocations[i] = min(prior_allocations_assignment[i], demands_realized[i]) if i!=n-1 else bundle_remaining
        bundle_remaining -= allocations[i]
    return allocations

In [28]:
## Tests
assert list(waterfilling_online_1(np.zeros(0), np.zeros(0), 5)) == []
assert list(waterfilling_online_1(np.array([1,2,3,4]), np.array([5,5,5,5]), 10)) == [1,2,3,4]
assert list(waterfilling_online_1(np.array([3,1,4,2]), np.array([2,3,2.5,1]), 8)) == [2,1,2.5,2.5]


In [29]:
## Online Water-filling algorithm where each agent solves waterfilling with realized current demand and expected following demands
def waterfilling_online_2(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        allocations[i] = waterfilling(np.append(demands_realized[i],demands_predicted[i+1:]), bundle_remaining)[0]
        bundle_remaining -= allocations[i]
    return allocations

In [30]:
def insert_sorted(lst, element):
    n = np.size(lst)
    if n==0:
        return np.array([element]),0
    if element<lst[0]:
        return np.append(element,lst),0
    if element>lst[n-1]:
        return np.append(lst,element),n
    left = 0
    right = n-1
    while left<right-1:
        mid_ind = int((left+right)/2)
        if element<lst[mid_ind]:
            right = mid_ind
        elif element > lst[mid_ind] :
            left = mid_ind
        if element == lst[mid_ind] or (element>lst[mid_ind] and element<lst[mid_ind+1]):
            return np.append(np.append(lst[:mid_ind+1],element),lst[mid_ind+1:]), mid_ind+1
    return np.append(np.append(lst[:left],element),lst[left:]), left

In [31]:
def delete_sorted(lst,element):
    n = np.size(lst)
    if element==lst[0]:
        return lst[1:]
    if element==lst[n-1]:
        return lst[:-1]
    left = 0
    right = n-1
    while left<right-1:
        mid_ind = int((left+right)/2)
        if element<lst[mid_ind]:
            right = mid_ind
        elif element > lst[mid_ind] :
            left = mid_ind
        else:
            return np.append(lst[:mid_ind],lst[mid_ind+1:])

In [36]:
## O(n^2) version of online algorithm that needs waterfilling evaluated multiple times
def waterfilling_dynamic(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    sorted_demands = np.sort(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        sorted_demands = delete_sorted(sorted_demands, demands_predicted[i])
        new_sorted_list,index = insert_sorted(sorted_demands,demands_realized[i])
        allocations[i] = (waterfilling_sorted(new_sorted_list, bundle_remaining))[index]
        bundle_remaining -= allocations[i]
    return allocations
    

In [37]:
## Tests 
assert list(waterfilling_online_2(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_online_2(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_online_2(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_online_2(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]

assert list(waterfilling_dynamic(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_dynamic(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_dynamic(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_dynamic(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]


In [38]:
## Online Water-filling algorithm where each agent is assigned infinite demand while finding optimal solution and allocation is readjusted
def waterfilling_online_3(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        future_allocations = waterfilling(np.append(np.Inf,demands_predicted[i+1:]), bundle_remaining)
        if future_allocations[0]>demands_realized[i] and i!=n-1:
            allocations[i] = demands_realized[i]
        else:
            allocations[i] = future_allocations[0]
        bundle_remaining -= allocations[i]
    return allocations

In [39]:
## Tests 
assert list(waterfilling_online_3(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_online_3(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_online_3(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_online_3(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]

In [99]:
# Bayes optimal version of water filling

def waterfilling_bayes_opt(town_demands,budget):
    n = np.size(town_demands)
    allocations = np.zeros(n)
    bundle_remaining = budget
    # Computes a random grid-size to use for the computation
    grid_size = budget / (5*n)
    # Computes othe Q_vals for the Bayes Optimal solution.
    Q_vals = bayes_opt(n, budget, grid_size)
    
    # For each town
    for i in range(n):
        # Compute the index of the remaining budget
        index = int(np.floor(bundle_remaining / grid_size))
        # Take the action that maximizes the Q function of their demands
        allocations[i] = np.argmax(Q_vals[index, 1:(index+1),(town_demands[i]-1).astype(int), i])*grid_size
        # Reduces the remaining budget
        bundle_remaining = bundle_remaining - allocations[i]
    return allocations

## Objective Function

In [42]:
## Calculate log of Nash welfare for objective function
def objective(demands, allocation):
    welfare_sum = 0
    for i in range(np.size(demands)):
        welfare_sum += np.log(min(1,allocation[i]/demands[i]))
    return welfare_sum

## Experiment

In [43]:
def make_demands_uniform_distribution(num_towns, demand_ranges):
    demands = np.zeros(num_towns)
    expected_demands = np.zeros(num_towns)
    for i in range(num_towns):
        demands[i] = np.random.uniform(0, demand_ranges[i])
        expected_demands[i] = demand_ranges[i]/2
    return demands, expected_demands


In [44]:
def make_demands_exponential_distribution(num_towns, demand_means):
    demands = np.zeros(num_towns)
    expected_demands = np.zeros(num_towns)
    for i in range(num_towns):
        demands[i] = np.random.exponential(demand_means[i])
        expected_demands[i] = demand_means[i]
    return demands, expected_demands


In [45]:
def make_demands_discrete_distribution(num_towns):
    demands = np.zeros(num_towns)
    expected_demands = np.zeros(num_towns)
    for i in range(num_towns):
        demands[i] = np.random.binomial(1,.5) + 1
        expected_demands[i] = 1.5
    return demands, expected_demands

In [46]:
num_iterations = 1
num_towns_range = 100
demands_max = 20
max_n = 1000
fix_num_towns = 10
max_b = 200


In [47]:
np.logspace(1,3).astype(int)

array([  10,   10,   12,   13,   14,   15,   17,   19,   21,   23,   25,
         28,   30,   33,   37,   40,   44,   49,   54,   59,   65,   71,
         79,   86,   95,  104,  115,  126,  138,  152,  167,  184,  202,
        222,  244,  268,  294,  323,  355,  390,  429,  471,  517,  568,
        625,  686,  754,  828,  910, 1000])

### Regret for Discrete Distribution of Demands

In [98]:
data_dict_1 = {'NumTowns':[],'Alg_1':[],'Alg_3':[], 'Alg_2':[]}

for n in np.logspace(1,2,10).astype(int):
    for i in range(num_iterations):
        data_dict_1['NumTowns'].append(n)
        town_demands, town_expected_demands = make_demands_discrete_distribution(n)
        budget = np.sum(town_expected_demands)

        opt = objective(town_demands, waterfilling(town_demands,budget))
        for j in range(3):
            if j==0:
                data_dict_1['Alg_1'].append(opt - objective(town_demands, waterfilling(town_expected_demands,budget)))
            if j==1:
                 data_dict_1['Alg_2'].append(opt - objective(town_demands, waterfilling_bayes_opt(town_demands,budget)))
            if j==2:
                data_dict_1['Alg_3'].append(opt - objective(town_demands, waterfilling_dynamic(town_expected_demands,town_demands,budget)))
df_uniform = pd.DataFrame(data_dict_1).melt(id_vars="NumTowns")
fig = px.scatter(df_uniform, x="NumTowns", y="value", color='variable')
fig.show()